<a href="https://colab.research.google.com/github/dhanushka365/SLEnergyConsumptionPrediction/blob/main/not_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

import numpy as np

from keras import callbacks
from keras import layers, models

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error

import xgboost as xgb
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [3]:
df_total = pd.read_csv("/content/sample_data/energy_dataset.csv", parse_dates=["time"],
                 date_parser=lambda date: pd.to_datetime(date).tz_convert('Europe/Madrid'), index_col="time")

df = df_total[["total load actual", "price actual"]]
df = df.rename(columns={"total load actual": "load", "price actual": "price"})
df

,load,price
time,,
2015-01-01 00:00:00+01:00,25385.0,65.41
2015-01-01 01:00:00+01:00,24382.0,64.92
2015-01-01 02:00:00+01:00,22734.0,64.48
2015-01-01 03:00:00+01:00,21286.0,59.32
2015-01-01 04:00:00+01:00,20264.0,56.04
...,...,...
2018-12-31 19:00:00+01:00,30653.0,77.02
2018-12-31 20:00:00+01:00,29735.0,76.16
2018-12-31 21:00:00+01:00,28071.0,74.30


In [5]:
time_column = df.index

min_datetime = time_column.min()
max_datetime = time_column.max()
hourly_datetimes = pd.date_range(min_datetime, max_datetime, freq="1H")

if np.sum(time_column == hourly_datetimes) == len(time_column):
    print("Times are correct.")

Times are correct.


In [6]:
def fill_nans(df, column_name):
    dfc = df[column_name]
    nans_num = np.sum(np.isnan(dfc))
        
    dfc = dfc.interpolate(method='linear', axis=0).ffill().bfill()
    df[column_name] = dfc
    
    print(f"Interpolated {nans_num} NaNs in column '{column_name}'.")

fill_nans(df, "price")
fill_nans(df, "load")

Interpolated 0 NaNs in column 'price'.
Interpolated 36 NaNs in column 'load'.


In [8]:
time_normalization = {
    "hour": lambda time: time / 24,
    "dayofweek": lambda time: time / 7,
    "month": lambda time: (time - 1) / 12,
}

datetimes = time_column.to_series().dt

for time_property, normalize_time_func in time_normalization.items():
    time = getattr(datetimes, time_property)
    time_normalized = normalize_time_func(time) 
    df[time_property] = time_normalized
    
df

,load,price,hour,dayofweek,month
time,,,,,
2015-01-01 00:00:00+01:00,25385.0,65.41,0.000000,0.428571,0.000000
2015-01-01 01:00:00+01:00,24382.0,64.92,0.041667,0.428571,0.000000
2015-01-01 02:00:00+01:00,22734.0,64.48,0.083333,0.428571,0.000000
2015-01-01 03:00:00+01:00,21286.0,59.32,0.125000,0.428571,0.000000
2015-01-01 04:00:00+01:00,20264.0,56.04,0.166667,0.428571,0.000000
...,...,...,...,...,...
2018-12-31 19:00:00+01:00,30653.0,77.02,0.791667,0.000000,0.916667
2018-12-31 20:00:00+01:00,29735.0,76.16,0.833333,0.000000,0.916667
2018-12-31 21:00:00+01:00,28071.0,74.30,0.875000,0.000000,0.916667


In [9]:
past_time_stamps = 24
target_time = 24
total_time_window = past_time_stamps + target_time

# 2016 is a leap year
train_split = (365 + 366) * 24
val_split = train_split + 365 * 24

X_attr = df[["load", "price"]].to_numpy()
# time properties are already normalized
X_time_norm = df[["hour", "dayofweek", "month"]].to_numpy()
Y_price = df[["price"]].to_numpy()
Y_load = df[["load"]].to_numpy()

# X will be shifted later
# shift Y values 48h ahead
X_attr_train = X_attr[:train_split]
X_time_train_norm = X_time_norm[:train_split]
Y_load_train = Y_load[total_time_window:train_split]
Y_price_train = Y_price[total_time_window:train_split]

X_attr_val = X_attr[train_split:val_split]
X_time_val_norm = X_time_norm[train_split:val_split]
Y_load_val = Y_load[train_split+total_time_window:val_split]
Y_price_val = Y_price[train_split+total_time_window:val_split]

X_attr_test = X_attr[val_split:]
X_time_test_norm = X_time_norm[val_split:]
Y_load_test = Y_load[val_split+total_time_window:]
Y_price_test = Y_price[val_split+total_time_window:]

X_scaler = MinMaxScaler(feature_range=(0, 1))
X_scaler.fit(X_attr_train)
X_attr_train_norm = X_scaler.transform(X_attr_train)
X_attr_val_norm = X_scaler.transform(X_attr_val)
X_attr_test_norm = X_scaler.transform(X_attr_test)

Y_load_scaler = MinMaxScaler(feature_range=(0, 1))
Y_load_scaler.fit(Y_load_train)
Y_load_train_norm = Y_load_scaler.transform(Y_load_train)
Y_load_val_norm = Y_load_scaler.transform(Y_load_val)
Y_load_test_norm = Y_load_scaler.transform(Y_load_test)

Y_price_scaler = MinMaxScaler(feature_range=(0, 1))
Y_price_scaler.fit(Y_price_train)
Y_price_train_norm = Y_price_scaler.transform(Y_price_train)
Y_price_val_norm = Y_price_scaler.transform(Y_price_val)
Y_price_test_norm = Y_price_scaler.transform(Y_price_test)

def to_time_windows(X):
    time_windows = []

    # use past measurements between 0..23h
    for i in range(past_time_stamps):
        time_windows.append(X[i:-total_time_window+i, :])

    time_windows = np.stack(time_windows, axis=1)
    return time_windows

# those will be used with the baseline models
X_attr_time_train_norm = np.concatenate([X_attr_train_norm, X_time_train_norm], axis=1)
X_attr_time_val_norm = np.concatenate([X_attr_val_norm, X_time_val_norm], axis=1)
X_attr_time_test_norm = np.concatenate([X_attr_test_norm, X_time_test_norm], axis=1)

X_attr_time_train_norm_windows = to_time_windows(X_attr_time_train_norm)
X_attr_time_val_norm_windows = to_time_windows(X_attr_time_val_norm)
X_attr_time_test_norm_windows = to_time_windows(X_attr_time_test_norm)

# those will be used with our model
X_attr_train_norm_windows = to_time_windows(X_attr_train_norm)
X_attr_val_norm_windows = to_time_windows(X_attr_val_norm)
X_attr_test_norm_windows = to_time_windows(X_attr_test_norm)

X_time_train_norm_values = X_time_train_norm[past_time_stamps:-target_time]
X_time_val_norm_values = X_time_val_norm[past_time_stamps:-target_time]
X_time_test_norm_values = X_time_test_norm[past_time_stamps:-target_time]

print("Train data", X_attr_time_train_norm_windows.shape, X_attr_train_norm_windows.shape, 
      X_time_train_norm_values.shape, Y_load_train_norm.shape, Y_price_train_norm.shape)
print("Validation data", X_attr_time_val_norm_windows.shape, X_attr_val_norm_windows.shape, 
      X_time_val_norm_values.shape, Y_load_val_norm.shape, Y_price_val_norm.shape)
print("Test data", X_attr_time_test_norm_windows.shape, X_attr_test_norm_windows.shape, 
      X_time_test_norm_values.shape, Y_load_test_norm.shape, Y_price_test_norm.shape)

Train data (17496, 24, 5) (17496, 24, 2) (17496, 3) (17496, 1) (17496, 1)
Validation data (8712, 24, 5) (8712, 24, 2) (8712, 3) (8712, 1) (8712, 1)
Test data (8712, 24, 5) (8712, 24, 2) (8712, 3) (8712, 1) (8712, 1)


In [10]:
n_features = X_attr_time_train_norm_windows.shape[2]

# source: https://www.kaggle.com/code/varanr/hourly-energy-demand-time-series-forecast/
def lstm_model():
    model = models.Sequential(name="LSTM")
    model.add(layers.LSTM(25, input_shape=(past_time_stamps, n_features)))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(1))

    return model

# source: https://www.kaggle.com/code/nicholasjhana/multi-variate-time-series-forecasting-tensorflow
def cnn_model():
    model = models.Sequential([
        layers.Conv1D(64, kernel_size=6, activation='relu', input_shape=(past_time_stamps, n_features)),
        layers.MaxPooling1D(2),
        layers.Conv1D(64, kernel_size=3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Flatten(),
        layers.Dropout(0.3),
        layers.Dense(128),
        layers.Dropout(0.3),
        layers.Dense(1)
    ], name="CNN")
    
    return model

# source: https://www.kaggle.com/code/dimitriosroussis/electricity-price-forecasting-with-dnns-eda
def run_xgboost(column_to_predict, x_train_norm, y_train_norm, x_val_norm, y_val_norm, x_test_norm, y_test_norm, y_scaler, y_val, y_test):    
    x_train_xgb = x_train_norm.reshape(-1, x_train_norm.shape[1] * x_train_norm.shape[2])
    x_val_xgb = x_val_norm.reshape(-1, x_val_norm.shape[1] * x_val_norm.shape[2])
    x_test_xgb = x_test_norm.reshape(-1, x_test_norm.shape[1] * x_test_norm.shape[2])
    
    param = {'eta': 0.03, 'max_depth': 180, 
             'subsample': 1.0, 'colsample_bytree': 0.95, 
             'alpha': 0.1, 'lambda': 0.15, 'gamma': 0.1,
             'objective': 'reg:linear', 'eval_metric': 'mae', 
             'silent': 1, 'min_child_weight': 0.1, 'n_jobs': -1}
    
    dtrain = xgb.DMatrix(x_train_xgb, y_train_norm)
    dval = xgb.DMatrix(x_val_xgb, y_val_norm)
    dtest = xgb.DMatrix(x_test_xgb, y_test_norm)
    eval_list = [(dtrain, 'train'), (dval, 'val')]
    
    evals_result = {}
    xgb_model = xgb.train(param, dtrain, 180, eval_list, evals_result=evals_result, verbose_eval=False)
    
    y_val_pred_norm = xgb_model.predict(dval)
    y_val_pred_norm = y_val_pred_norm.reshape(-1, 1)
    y_val_pred = y_scaler.inverse_transform(y_val_pred_norm)
    mae_val = mean_absolute_error(y_val, y_val_pred)
    print('XGBoost - MAE on validation dataset', round(mae_val, 2))
    
    y_test_pred_norm = xgb_model.predict(dtest)
    y_test_pred_norm = y_test_pred_norm.reshape(-1, 1)
    y_test_pred = y_scaler.inverse_transform(y_test_pred_norm)
    mae_test = mean_absolute_error(y_test, y_test_pred)
    print('XGBoost - MAE on test dataset', round(mae_test, 2))
    
    history = {
        "loss": evals_result['train']['mae'],
        "val_loss": evals_result['val']['mae']
    }

    plot_loss(history)
    plot_prediction(column_to_predict, y_val, y_val_pred)
    
lstm_load_model = lstm_model()
lstm_price_model = lstm_model()
lstm_price_model.summary()

print("\n\n")
cnn_load_model = cnn_model()
cnn_price_model = cnn_model()
cnn_price_model.summary()

Model: "LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 25)                3100      
                                                                 
 dropout_1 (Dropout)         (None, 25)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 26        
                                                                 
Total params: 3,126
Trainable params: 3,126
Non-trainable params: 0
_________________________________________________________________



Model: "CNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 19, 64)            1984      
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 9, 64) 

In [11]:
def gru_encoder(time_series):
    x = layers.GRU(units=16)(time_series)

    return [x, "GRU"]


def cnn_encoder(time_series): 
    num_filters = 8
       
    x = layers.Conv1D(num_filters * 1, 3, activation="relu", padding="same")(time_series)
    x = layers.Conv1D(num_filters * 2, 3, activation="relu", padding="same", strides=2)(x)
    x = layers.Conv1D(num_filters * 2, 3, activation="relu", padding="same")(x)
    x = layers.Conv1D(num_filters * 3, 3, activation="relu", padding="same", strides=2)(x)
    x = layers.Conv1D(num_filters * 3, 3, activation="relu", padding="same")(x)
    x = layers.Conv1D(num_filters * 4, 3, activation="relu", padding="same", strides=2)(x)
    x = layers.Conv1D(num_filters * 4, 3, activation="relu", padding="same")(x)
    x = layers.Flatten()(x)
    
    return [x, "CNN"]


def conv_encoder(time_series):
    x = tf.transpose(time_series, [0, 2, 1])
    x = layers.Conv1D(16, 3, padding="same", activation="linear")(x)
    x = layers.Conv1D(8, 3, padding="same", activation="linear")(x)
    x = layers.Conv1D(4, 3, padding="same", activation="linear")(x)
    x = layers.Flatten()(x)
    
    return [x, "CONV"]


def concat_model(feature_encoder):
    # past 24 timestamps of load and price
    time_windows = layers.Input((past_time_stamps, 2))
    # hour, day of the week, month of the time stamp when the forecast happens
    time_values = layers.Input((3))
    
    x, encoder_name = feature_encoder(time_windows)
    y = layers.Dense(16, activation="relu")(time_values)
    x = layers.Concatenate()([x, y])
    
    x = layers.Dense(16, activation="relu")(x)
    x = layers.Dropout(rate=0.1)(x)
    x = layers.Dense(1)(x)

    model = models.Model(inputs = [time_windows, time_values], outputs=x, 
                         name=f"{encoder_name}_concat_time")
    
    return model

gru_concat_time_load_model = concat_model(gru_encoder)
gru_concat_time_price_model = concat_model(gru_encoder)
gru_concat_time_price_model.summary()

print("\n\n")
cnn_concat_time_load_model = concat_model(cnn_encoder)
cnn_concat_time_price_model = concat_model(cnn_encoder)
cnn_concat_time_price_model.summary()

print("\n\n")
conv_concat_time_load_model = concat_model(conv_encoder)
conv_concat_time_price_model = concat_model(conv_encoder)
conv_concat_time_price_model.summary()

Model: "GRU_concat_time"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 24, 2)]      0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 3)]          0           []                               
                                                                                                  
 gru_1 (GRU)                    (None, 16)           960         ['input_3[0][0]']                
                                                                                                  
 dense_9 (Dense)                (None, 16)           64          ['input_4[0][0]']                
                                                                                    

In [12]:
def plot_loss(history):
    plt.plot(history["loss"])
    plt.plot(history["val_loss"])
    plt.title('Model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['training', 'validation'], loc='upper right')
    plt.show()

two_weeks = 24 * 14
x = time_column.to_series().to_numpy()[train_split+total_time_window:train_split+total_time_window+two_weeks]
    
def plot_prediction(column_to_predict, y_true, y_pred):
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%d.%m.%Y %H:%M', tz=time_column.tz))
    plt.title(f"Electricity {column_to_predict}")
    plt.plot(x, y_true[:two_weeks])
    plt.plot(x, y_pred[:two_weeks])
    plt.legend(['ground truth', 'prediction'], loc='upper left')
    plt.gcf().autofmt_xdate()
    plt.show()

In [13]:
epochs = 200
# 0 - no output, 1 - detailed output, 2 - brief output
verbosity = 0

def run_model(model, column_to_predict, x_train_norm, y_train_norm, x_val_norm, y_val_norm, x_test_norm, y_scaler, y_val, y_test):
    weights_path = f"{model.name}_{column_to_predict}_weights.h5"

    modelckpt_callback = callbacks.ModelCheckpoint(
        monitor="val_loss",
        filepath=weights_path,
        verbose=verbosity,
        save_weights_only=True,
        save_best_only=True,
    )
    
    model.compile(loss='mae', optimizer="adam")

    history = model.fit(x_train_norm, y_train_norm, epochs=epochs, 
              validation_data=(x_val_norm, y_val_norm),
              callbacks=[modelckpt_callback],
              verbose=verbosity)
    
    model.load_weights(weights_path)

    y_val_pred_norm = model.predict(x_val_norm)
    y_val_pred = y_scaler.inverse_transform(y_val_pred_norm)

    mae_val = mean_absolute_error(y_val, y_val_pred)
    print(f"{model.name} - MAE on validation dataset", round(mae_val, 2))

    y_test_pred_norm = model.predict(x_test_norm)
    y_test_pred = y_scaler.inverse_transform(y_test_pred_norm)

    mae_test = mean_absolute_error(y_test, y_test_pred)
    print(f"{model.name} - MAE on test dataset", round(mae_test, 2))
    
    plot_loss(history.history)
    plot_prediction(column_to_predict, y_val, y_val_pred)

In [ ]:
run_model(lstm_load_model, "load", 
          X_attr_time_train_norm_windows, Y_load_train_norm, 
          X_attr_time_val_norm_windows, Y_load_val_norm, 
          X_attr_time_test_norm_windows, Y_load_scaler, Y_load_val, Y_load_test)


In [ ]:
run_model(cnn_load_model, "load",
          X_attr_time_train_norm_windows, Y_load_train_norm, 
          X_attr_time_val_norm_windows, Y_load_val_norm, 
          X_attr_time_test_norm_windows, Y_load_scaler, Y_load_val, Y_load_test)

In [ ]:
run_xgboost("load", 
            X_attr_time_train_norm_windows, Y_load_train_norm, 
            X_attr_time_val_norm_windows, Y_load_val_norm, 
            X_attr_time_test_norm_windows, Y_load_test_norm, 
            Y_load_scaler, Y_load_val, Y_load_test)

In [ ]:
run_model(gru_concat_time_load_model, "load",
          [X_attr_train_norm_windows, X_time_train_norm_values], Y_load_train_norm, 
          [X_attr_val_norm_windows, X_time_val_norm_values], Y_load_val_norm, 
          [X_attr_test_norm_windows, X_time_test_norm_values], Y_load_scaler, Y_load_val, Y_load_test)

In [ ]:
run_model(cnn_concat_time_load_model, "load",
          [X_attr_train_norm_windows, X_time_train_norm_values], Y_load_train_norm, 
          [X_attr_val_norm_windows, X_time_val_norm_values], Y_load_val_norm, 
          [X_attr_test_norm_windows, X_time_test_norm_values], Y_load_scaler, Y_load_val, Y_load_test)

In [ ]:
run_model(lstm_price_model, "price", 
          X_attr_time_train_norm_windows, Y_price_train_norm, 
          X_attr_time_val_norm_windows, Y_price_val_norm, 
          X_attr_time_test_norm_windows, Y_price_scaler, Y_price_val, Y_price_test)

In [ ]:
run_model(cnn_price_model, "price",
          X_attr_time_train_norm_windows, Y_price_train_norm, 
          X_attr_time_val_norm_windows, Y_price_val_norm, 
          X_attr_time_test_norm_windows, Y_price_scaler, Y_price_val, Y_price_test)

In [ ]:
run_xgboost("price", 
            X_attr_time_train_norm_windows, Y_price_train_norm, 
            X_attr_time_val_norm_windows, Y_price_val_norm, 
            X_attr_time_test_norm_windows, Y_price_test_norm, 
            Y_price_scaler, Y_price_val, Y_price_test)

In [ ]:
run_model(gru_concat_time_price_model, "price",
          [X_attr_train_norm_windows, X_time_train_norm_values], Y_price_train_norm, 
          [X_attr_val_norm_windows, X_time_val_norm_values], Y_price_val_norm, 
          [X_attr_test_norm_windows, X_time_test_norm_values], Y_price_scaler, Y_price_val, Y_price_test)

In [ ]:
run_model(cnn_concat_time_price_model, "price",
          [X_attr_train_norm_windows, X_time_train_norm_values], Y_price_train_norm, 
          [X_attr_val_norm_windows, X_time_val_norm_values], Y_price_val_norm, 
          [X_attr_test_norm_windows, X_time_test_norm_values], Y_price_scaler, Y_price_val, Y_price_test)

In [ ]:
run_model(conv_concat_time_price_model, "price",
          [X_attr_train_norm_windows, X_time_train_norm_values], Y_price_train_norm, 
          [X_attr_val_norm_windows, X_time_val_norm_values], Y_price_val_norm, 
          [X_attr_test_norm_windows, X_time_test_norm_values], Y_price_scaler, Y_price_val, Y_price_test)